# Datashading LandSat8 raster satellite imagery

Datashader is fundamentally a rasterizing library, turning data into rasters (image-like arrays), but it is also useful for already-rasterized data like satellite imagery.  For raster data, datashader uses the separate [rasterio](https://mapbox.github.io/rasterio) library to re-render the data to whatever new bounding box and resolution the user requests, and the rest of the datashader pipeline can then be used to visualize and analyze the data.  This demo shows how to work with a set of raster satellite data, generating images as needed and overlaying them on geographic coordinates.

In [ ]:
import os

import numpy as np
import xarray as xr

from bokeh.plotting import Figure
from bokeh.io import output_notebook

from datashader.bokeh_ext import InteractiveImage
import datashader as ds
import datashader.transfer_functions as tf

from bokeh.tile_providers import STAMEN_TONER

output_notebook()

### Load LandSat Data 

LandSat data is measured in different frequency bands, revealing different types of information:

```
Band      Wavelength     Resolution   Description
          (micrometers)  (meters)

Band 1     0.43 - 0.45     30         Coastal aerosol
Band 2     0.45 - 0.51     30         Blue
Band 3     0.53 - 0.59     30         Green
Band 4     0.64 - 0.67     30         Red
Band 5     0.85 - 0.88     30         Near Infrared (NIR)
Band 6     1.57 - 1.65     30         SWIR 1
Band 7     2.11 - 2.29     30         SWIR 2
Band 8     0.50 - 0.68     15         Panchromatic
Band 9     1.36 - 1.38     30         Cirrus
Band 10   10.60 - 11.19   100 * (30)  Thermal Infrared (TIRS) 1
Band 11   11.50 - 12.51   100 * (30)  Thermal Infrared (TIRS) 2
```

In [ ]:
data_dir = './data'

band1 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B1.TIF'))
band2 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B2.TIF'))
band3 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B3.TIF'))
band4 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B4.TIF'))
band5 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B5.TIF'))
band6 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B6.TIF'))
band7 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B7.TIF'))
band8 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B8.TIF'))
band9 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B9.TIF'))
band10 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B10.TIF'))
band11 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_B11.TIF'))
band12 = xr.open_rasterio(os.path.join(data_dir, 'MERCATOR_LC80210392016114LGN00_BQA.TIF'))
# Notice the MERCATOR prefix which indicates the data was project to Mercator CRS

res = ds.utils.calc_res(band1)
xmin, ymin, xmax, ymax = ds.utils.calc_bbox(band1.x.values, band1.y.values, res)

# Declare value used in the above files for no-data values, as bandX.nodata was not filled in
nodata=1

### Define geographic plot

We'll plot this data using Bokeh, overlaying it on map tiles for context:

In [ ]:
def base_plot(tools='pan,wheel_zoom,reset',plot_width=720, plot_height=400, x_range=None, y_range=None, **plot_args):
    p = Figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        background_fill_color='black',
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.add_tile(STAMEN_TONER)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

## Rendering LandSat data as images

The bands measured by LandSat include wavelengths covering the visible spectrum, but also other ranges, and so it's possible to visualize this data in many different ways, in both true color (using the visible spectrum directly) or false color (usually showing other bands).  Some examples are shown in the sections below.

### Just the Blue Band

Using datashader's default histogram-equalized colormapping, the full range of data is visible in the plot:

In [ ]:
def update_image(x_range, y_range, w, h, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.raster(band2, layer=0)
    blue_img = tf.shade(agg,cmap=['black','white'])
    return blue_img

p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, update_image)

You will usually want to zoom in, which will re-rasterize the image if you are in a live notebook, and then re-equalize the colormap to show all the detail available.  If you are on a static copy of the notebook, only the original resolution at which the image was rendered will be available, but zooming will still update the map tiles to whatever resolution is requested.

The plots below use a different type of colormap processing, implemented as a custom transfer function:

In [ ]:
from datashader.utils import ngjit

@ngjit
def normalize_data(agg):
    out = np.zeros_like(agg)
    min_val = 0
    max_val = 2**16 - 1
    range_val = max_val - min_val
    col, rows = agg.shape
    c = 40
    th = .125
    for x in range(col):
        for y in range(rows):
            val = agg[x, y]
            norm = (val - min_val) / range_val
            norm = 1 / (1 + np.exp(c * (th - norm))) # bonus
            out[x, y] = norm * 255.0
    return out

In [ ]:
def combine_bands(r, g, b):
    a = (np.where(np.logical_or(np.isnan(r),r<=nodata),0,255)).astype(np.uint8)    
    r = (normalize_data(r)).astype(np.uint8)
    g = (normalize_data(g)).astype(np.uint8)
    b = (normalize_data(b)).astype(np.uint8)
    col, rows = r.shape
    img = np.dstack([r, g, b, a]).view(np.uint32).reshape(r.shape)
    return tf.Image(data=img[::-1])

### True Color (Red=Red, Green=Green, Blue=Blue)

Mapping the Red, Green, and Blue bands to the R, G, and B channels of an image reconstructs the image as it would appear to an ordinary camera from that viewpoint:

In [ ]:
def true_color(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b, layer=0).data for b in (band4, band3, band2)]
    return combine_bands(r, g, b)

p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, true_color)

Other combinations highlight particular features of interest based on the different spectral properties of reflectances from various objects and surfaces:

### Color Infrared (Vegetation) (Red=Near Infrared, Green=Red, Blue=Green)

In [ ]:
def color_infrared(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b, layer=0).data for b in (band5, band4, band3)]
    return combine_bands(r, g, b)

p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, color_infrared)

### False Color (Urban) (Red=SWIR 2, Green=SWIR 1, Blue=Red)

In [ ]:
def false_color_urban(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b, layer=0).data for b in (band7, band6, band4)]
    return combine_bands(r, g, b)

p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, false_color_urban)

### False Color 2 (Red=Near Infrared, Green=SWIR 1, Blue=Coastal)

In [ ]:
def false_color_veg(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b, layer=0).data for b in (band5, band7, band1)]
    return combine_bands(r, g, b)

p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, false_color_veg)

### Land vs. Water (Red=Near Infrared, Green=SWIR 1, Blue=Red)

In [ ]:
def land_vs_water(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b, layer=0).data for b in (band5, band6, band4)]
    return combine_bands(r, g, b)

p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, land_vs_water)

### Shortwave Infrared (Red=SWIR2, Green=Near Infrared, Blue=Red)

In [ ]:
def shortwave_infrared(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    r, g, b = [cvs.raster(b, layer=0).data for b in (band7, band5, band4)]
    return combine_bands(r, g, b)

p = base_plot(x_range=(xmin, xmax), y_range=(ymin, ymax))
InteractiveImage(p, shortwave_infrared)

All the various ways of combining aggregates supported by [xarray](http://xarray.pydata.org) are available for these channels, making it simple to make your own custom visualizations highlighting any combination of bands that reveal something of interest.